# Player Stats Collecting 
(Don't need to collect player stats for now)

In [1]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
import time


def get_page_content(url,encoding='UTF-8',remove_linebreaks=True
                    ,header = {'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                            r'Chrome/41.0.2227.1 Safari/537.36'}):
    session = requests.Session()
    base_url = url
    response = session.get(base_url, headers=header)
    response.encoding = encoding
    page_content = BeautifulSoup(response.content, "html.parser")
    page_content_str = str(page_content)
    if remove_linebreaks:
        page_content_str = re.sub('\n','',page_content_str)
    return page_content_str

# All Games Collecting
* Get the schedule from http://cba.sports.sina.com.cn/cba/schedule/all/?qleagueid=205&qmonth=&qteamid=
* Get all the links
* Loop through all the game links and scrape detailed stats

### To do:
* get all values in "select" tag, just in case this base url does not work anymore
* get all links
* after get all links, scrape each game's stats
* 暂定 主队_link，统计_link这样的header, 后期可能单独建一个表放所有entity（球队，球员）的link
* automatically determine number of columns in the table
* Not sure if I want to rescrape all the data every week or just go look for the missing game links using known data
* 单节得分，教练，领队可以记在game表里

In [2]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
import pandas as pd
import datetime
import re

header = {'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                        r'Chrome/41.0.2227.1 Safari/537.36'}
session = requests.Session()
base_url = r"http://cba.sports.sina.com.cn/cba/schedule/all/?qleagueid=205&qmonth=&qteamid="
response = session.get(base_url, headers=header)

response.encoding = 'GB2312'

page_content = BeautifulSoup(response.content, "html.parser")
page_content_str = str(page_content)
page_content_str = re.sub('\n','',page_content_str)

In [3]:
# Get table division
table_content = re.findall('<div class="blk_wrap"><table>(.*)</table></div>',page_content_str)

len(table_content)

1

In [4]:
table_content_str = table_content[0]
table_content_str[:1000]

'<thead><tr class="even"><th>轮次</th><th>日期</th><th>主队</th><th>比分</th><th>客队</th><th>战报</th><th>统计</th><th>组图</th><th>地点</th><th>电视</th></tr></thead><tbody><tr class="odd"><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t第1轮\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td>2019-11-01\xa019:35</td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/1/" target="_blank">                            广东                            </a></td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t107:98\t\t\t\t\t\t\t\t\t\t\t\t\t\t</a></td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/9/" target="_blank">                            辽宁                            </a></td><td><a href="http://sports.sina.com.cn/basketball/cba/2019-11-01/doc-iicezzrr6602072.shtml" target="_blank">战报</a></td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">统计</a></td><td><a href="http://slide.sports.sina.com.cn/c

### Get table headers

In [5]:
# Get headers


table_headers = re.findall('<th>(.*?)</th>',table_content_str)
table_headers

['轮次', '日期', '主队', '比分', '客队', '战报', '统计', '组图', '地点', '电视']

### Get table rows (table body, 也就是赛程表)

In [6]:
# Get table data



table_body = re.findall('<tbody>(.*?)</tbody>',table_content_str)
len(table_body)

1

In [7]:
table_body_str = table_body[0]
# get rid of invalid character
table_body_str = table_body_str.replace(u'\xa0',' ')
table_body_str[:1000]

'<tr class="odd"><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t第1轮\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td>2019-11-01 19:35</td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/1/" target="_blank">                            广东                            </a></td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t107:98\t\t\t\t\t\t\t\t\t\t\t\t\t\t</a></td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/9/" target="_blank">                            辽宁                            </a></td><td><a href="http://sports.sina.com.cn/basketball/cba/2019-11-01/doc-iicezzrr6602072.shtml" target="_blank">战报</a></td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">统计</a></td><td><a href="http://slide.sports.sina.com.cn/cba/slide_2_792_233270.html" target="_blank">组图</a></td><td>东莞</td><td></td></tr><tr class="even"><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t第1轮\t\t\t\t\t\t\t\t\t\t\t\t\

In [8]:
# check last 1000 characters
table_body_str[-1000:]

'                     山东                            </a></td><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t--\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18839/" target="_blank">统计</a></td><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t--\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td>晋江</td><td></td></tr><tr class="even"><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t第46轮\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td>2020-03-12 19:35</td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/11/" target="_blank">                            新疆                            </a></td><td><a href="http://cba.sports.sina.com.cn/cba/schedule/show/18840/" target="_blank">                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tVS\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</a></td><td><a href="http://cba.sports.sina.com.cn/cba/team/show/113/" target="_blank">                            天津                            </a></td><td>\t\t\t\t\t\t\t\t\t\t\t\t\t\t--\t\t\t\t\t\t\t\t\t\t\t\t\t\t</td><td><a 

In [9]:
table_cells = re.findall('<td>(.*?)</td>',table_body_str)

# CBA has 46 rounds, each round constits of 10 games, so we need to have 46*10*10 cells = 4600 cells
len(table_cells)

4600

In [10]:
# get rid of spaces
table_cells = list(map(lambda x: x.strip(),table_cells))
table_cells[:10]

['第1轮',
 '2019-11-01 19:35',
 '<a href="http://cba.sports.sina.com.cn/cba/team/show/1/" target="_blank">                            广东                            </a>',
 '<a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">                            \t\t\t\t\t\t\t\t\t\t\t\t\t\t107:98\t\t\t\t\t\t\t\t\t\t\t\t\t\t</a>',
 '<a href="http://cba.sports.sina.com.cn/cba/team/show/9/" target="_blank">                            辽宁                            </a>',
 '<a href="http://sports.sina.com.cn/basketball/cba/2019-11-01/doc-iicezzrr6602072.shtml" target="_blank">战报</a>',
 '<a href="http://cba.sports.sina.com.cn/cba/schedule/show/18381/" target="_blank">统计</a>',
 '<a href="http://slide.sports.sina.com.cn/cba/slide_2_792_233270.html" target="_blank">组图</a>',
 '东莞',
 '']

### Get actual texts and links

In [11]:
# get actual text
table_cells_txt = list(map(lambda x: re.findall('<a.*?>(.*)</a>',x)[0].strip() if '<a' in x else x, table_cells))
print(len(table_cells_txt))
print(table_cells_txt[:20])
# get all the links
table_cells_links = list(map(lambda x: re.findall('<a href="(.*?)" target="_blank">'
                                                  ,x)[0].strip() if '<a' in x else None, table_cells))
print(len(table_cells_links))
print(table_cells_links[:20])

4600
['第1轮', '2019-11-01 19:35', '广东', '107:98', '辽宁', '战报', '统计', '组图', '东莞', '', '第1轮', '2019-11-02 19:35', '江苏', '96:97', '浙江', '战报', '统计', '组图', '苏州', '']
4600
[None, None, 'http://cba.sports.sina.com.cn/cba/team/show/1/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://cba.sports.sina.com.cn/cba/team/show/9/', 'http://sports.sina.com.cn/basketball/cba/2019-11-01/doc-iicezzrr6602072.shtml', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://slide.sports.sina.com.cn/cba/slide_2_792_233270.html', None, None, None, None, 'http://cba.sports.sina.com.cn/cba/team/show/2/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18382/', 'http://cba.sports.sina.com.cn/cba/team/show/6/', 'http://sports.sina.com.cn/basketball/cba/2019-11-02/doc-iicezuev6752945.shtml', 'http://cba.sports.sina.com.cn/cba/schedule/show/18382/', 'http://slide.sports.sina.com.cn/cba/slide_2_786_233393.html', None, None]


### Save data as schedule table

In [12]:
# transform into 

import numpy as np
table_cells_txt = np.reshape(table_cells_txt,[-1,10]).tolist()
table_cells_links = np.reshape(table_cells_links,[-1,10]).tolist()
print(table_cells_txt[:10])
print(table_cells_links[:10])

[['第1轮', '2019-11-01 19:35', '广东', '107:98', '辽宁', '战报', '统计', '组图', '东莞', ''], ['第1轮', '2019-11-02 19:35', '江苏', '96:97', '浙江', '战报', '统计', '组图', '苏州', ''], ['第1轮', '2019-11-02 19:35', '四川', '87:134', '新疆', '战报', '统计', '组图', '成都', ''], ['第1轮', '2019-11-02 19:35', '山东', '108:95', '八一', '战报', '统计', '组图', '济南', ''], ['第1轮', '2019-11-02 19:35', '北控', '108:103', '深圳', '战报', '统计', '组图', '北京', ''], ['第1轮', '2019-11-02 19:35', '青岛', '103:105', '吉林', '战报', '统计', '组图', '青岛', ''], ['第1轮', '2019-11-02 19:35', '广厦', '93:90', '福建', '战报', '统计', '组图', '诸暨', ''], ['第1轮', '2019-11-02 20:00', '上海', '93:84', '同曦', '战报', '统计', '组图', '上海', ''], ['第1轮', '2019-11-03 19:35', '天津', '81:103', '北京', '战报', '统计', '组图', '天津', ''], ['第1轮', '2019-11-03 19:35', '广州', '96:123', '山西', '战报', '统计', '组图', '广州', '']]
[[None, None, 'http://cba.sports.sina.com.cn/cba/team/show/1/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://cba.sports.sina.com.cn/cba/team/show/9/', 'http://sports.sina.com.cn/basketball/

In [13]:
df_schedule_text = pd.DataFrame(data = table_cells_txt,columns=table_headers)
df_schedule_links = pd.DataFrame(data = table_cells_links,columns=[header+'_link' for header in table_headers])

In [14]:
df_schedule_text.head()

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视
0,第1轮,2019-11-01 19:35,广东,107:98,辽宁,战报,统计,组图,东莞,
1,第1轮,2019-11-02 19:35,江苏,96:97,浙江,战报,统计,组图,苏州,
2,第1轮,2019-11-02 19:35,四川,87:134,新疆,战报,统计,组图,成都,
3,第1轮,2019-11-02 19:35,山东,108:95,八一,战报,统计,组图,济南,
4,第1轮,2019-11-02 19:35,北控,108:103,深圳,战报,统计,组图,北京,


In [15]:
df_schedule_links.head()

,轮次_link,日期_link,主队_link,比分_link,客队_link,战报_link,统计_link,组图_link,地点_link,电视_link
0,None,None,http://cba.sports.sina.com.cn/cba/team/show/1/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/9/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,None,None
1,None,None,http://cba.sports.sina.com.cn/cba/team/show/2/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/6/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,None,None
2,None,None,http://cba.sports.sina.com.cn/cba/team/show/182/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/11/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,None,None
3,None,None,http://cba.sports.sina.com.cn/cba/team/show/15/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/3/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,None,None
4,None,None,http://cba.sports.sina.com.cn/cba/team/show/369/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/8/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,None,None


In [16]:
df_schedule_full = pd.merge(df_schedule_text,df_schedule_links,left_index=True,right_index=True)
# remove columns that are all None
df_schedule_full.dropna(axis=1,how='all',inplace=True)
df_schedule_full.head()

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视,主队_link,比分_link,客队_link,战报_link,统计_link,组图_link
0,第1轮,2019-11-01 19:35,广东,107:98,辽宁,战报,统计,组图,东莞,,http://cba.sports.sina.com.cn/cba/team/show/1/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/9/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...
1,第1轮,2019-11-02 19:35,江苏,96:97,浙江,战报,统计,组图,苏州,,http://cba.sports.sina.com.cn/cba/team/show/2/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/6/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...
2,第1轮,2019-11-02 19:35,四川,87:134,新疆,战报,统计,组图,成都,,http://cba.sports.sina.com.cn/cba/team/show/182/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/11/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...
3,第1轮,2019-11-02 19:35,山东,108:95,八一,战报,统计,组图,济南,,http://cba.sports.sina.com.cn/cba/team/show/15/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/3/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...
4,第1轮,2019-11-02 19:35,北控,108:103,深圳,战报,统计,组图,北京,,http://cba.sports.sina.com.cn/cba/team/show/369/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/8/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...


In [17]:
# id consists of season and a 5-digit label of the game - 2019-2020 season, game No.1 = 2019202000001
# lable is generated by unique http address of the game stats
uid_column = ['20192020'+f'{item+1:05d}' for item in pd.factorize(df_schedule_full['统计_link'])[0].tolist()]
df_schedule_full['UID'] = uid_column
# get rid of Hanzi and convert into integers
df_schedule_full['轮次'] = df_schedule_full['轮次'].apply(lambda x: int(re.findall('(\d+)',x)[0]))

In [18]:
df_schedule_full.head()

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视,主队_link,比分_link,客队_link,战报_link,统计_link,组图_link,UID
0,1,2019-11-01 19:35,广东,107:98,辽宁,战报,统计,组图,东莞,,http://cba.sports.sina.com.cn/cba/team/show/1/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/9/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,2019202000001
1,1,2019-11-02 19:35,江苏,96:97,浙江,战报,统计,组图,苏州,,http://cba.sports.sina.com.cn/cba/team/show/2/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/6/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000002
2,1,2019-11-02 19:35,四川,87:134,新疆,战报,统计,组图,成都,,http://cba.sports.sina.com.cn/cba/team/show/182/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/11/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000003
3,1,2019-11-02 19:35,山东,108:95,八一,战报,统计,组图,济南,,http://cba.sports.sina.com.cn/cba/team/show/15/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/3/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000004
4,1,2019-11-02 19:35,北控,108:103,深圳,战报,统计,组图,北京,,http://cba.sports.sina.com.cn/cba/team/show/369/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/8/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,2019202000005


In [19]:
df_schedule_full

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视,主队_link,比分_link,客队_link,战报_link,统计_link,组图_link,UID
0,1,2019-11-01 19:35,广东,107:98,辽宁,战报,统计,组图,东莞,,http://cba.sports.sina.com.cn/cba/team/show/1/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/9/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,2019202000001
1,1,2019-11-02 19:35,江苏,96:97,浙江,战报,统计,组图,苏州,,http://cba.sports.sina.com.cn/cba/team/show/2/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/6/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000002
2,1,2019-11-02 19:35,四川,87:134,新疆,战报,统计,组图,成都,,http://cba.sports.sina.com.cn/cba/team/show/182/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/11/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000003
3,1,2019-11-02 19:35,山东,108:95,八一,战报,统计,组图,济南,,http://cba.sports.sina.com.cn/cba/team/show/15/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/3/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000004
4,1,2019-11-02 19:35,北控,108:103,深圳,战报,统计,组图,北京,,http://cba.sports.sina.com.cn/cba/team/show/369/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/8/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,2019202000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,46,2020-03-12 19:35,辽宁,VS,江苏,--,统计,--,沈阳,,http://cba.sports.sina.com.cn/cba/team/show/9/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/2/,None,http://cba.sports.sina.com.cn/cba/schedule/sho...,None,2019202000456
456,46,2020-03-12 19:35,八一,VS,青岛,--,统计,--,南昌,,http://cba.sports.sina.com.cn/cba/team/show/3/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/114/,None,http://cba.sports.sina.com.cn/cba/schedule/sho...,None,2019202000457
457,46,2020-03-12 19:35,吉林,VS,广州,--,统计,--,长春,,http://cba.sports.sina.com.cn/cba/team/show/12/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/13/,None,http://cba.sports.sina.com.cn/cba/schedule/sho...,None,2019202000458
458,46,2020-03-12 19:35,福建,VS,山东,--,统计,--,晋江,,http://cba.sports.sina.com.cn/cba/team/show/7/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/15/,None,http://cba.sports.sina.com.cn/cba/schedule/sho...,None,2019202000459


In [20]:
df_schedule_full.to_csv(f'StatsData/GamesSchedulePage_{datetime.date.today()}.csv', index=False)

## Details Page

In [21]:
links_list = list(df_schedule_full['统计_link'])
links_list

['http://cba.sports.sina.com.cn/cba/schedule/show/18381/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18382/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18383/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18384/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18385/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18387/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18388/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18386/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18389/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18390/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18391/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18392/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18393/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18394/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18395/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18396/',
 'http://cba.sports.sina.com.cn/cba/schedule/show/18397/

In [22]:
page_content_str = get_page_content(r'http://cba.sports.sina.com.cn/cba/schedule/show/18381/'
                                    ,encoding='GB2312')
page_content_str

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><title>广东VS辽宁_CBA联赛单场比分_中国篮球数据库_篮球-CBA_新浪体育_新浪网</title><meta content="广东VS辽宁CBA联赛赛程赛果，CBA联赛直播，CBA联赛赛程赛果，CBA联赛直播，中国篮球数据库" name="keywords"/><meta content="新浪CBA联赛，提供CBA联赛最新赛程赛果、CBA联赛直播、CBA联赛总决赛、CBA联赛录像、CBA联赛视频集锦、CBA联赛球队球员、CBA联赛十佳球、CBA联赛快讯、CBA联赛图片等丰富的CBA联赛新闻和数据。" name="description"/><link href="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/base.css" rel="stylesheet" type="text/css"/><link href="http://n.sinaimg.cn/default/7919a7ba/20190305/score.css" rel="stylesheet" type="text/css"/><script src="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/cba.js" type="text/javascript"></script></head><body><!-- wrap begin --><div class="wrap" id="wraper"><style type="text/css">\t.sHBorder {    border: 1px solid #E3E3E3;    overflow: hidden;    padding: 5px 

###  每节比分对比

In [23]:
hometeam_name = re.findall('<span class="team_name team_nameA" style="font-size:15px">(.*?)</span>'
                            ,page_content_str)[0]
awayteam_name = re.findall('<span class="team_name team_nameB" style="font-size:15px">(.*?)</span>'
                            ,page_content_str)[0]
hometeam_score = re.findall('<div class="team_score team_scoreA"><span>(\d*?)</span></div>',page_content_str)[0]
awayteam_score = re.findall('<div class="team_score team_scoreB"><span>(\d*?)</span></div>',page_content_str)[0]
print(hometeam_name,hometeam_score)
print(awayteam_name,awayteam_score)

广东 107
辽宁 98


In [24]:
# <div class="score_pad"><div class="scores"></div>
pts_qt_headers = re.findall('<div class="scores">(.*?)<div class="score_tab">',page_content_str)
pts_qt_headers_str = pts_qt_headers[0]

pts_qt_headers_lst = re.findall('<span.*?>(.*?)</span>',pts_qt_headers_str)
pts_qt_headers_lst = pts_qt_headers_lst[1:-1]
pts_qt_headers_lst

['第一节', '第二节', '第三节', '第四节', '加时一', '加时二', '加时三']

In [25]:
# <div class="score_pad"><div class="scores"></div>
pts_qt_pts = re.findall('<div class="team_score team_scoreA">.*?</div>(.*?)<div class="team_score team_scoreB">'
                        ,page_content_str)
pts_qt_pts_str = pts_qt_pts[0]
pts_qt_pts_str = pts_qt_pts_str.replace('\xa0','')
pts_qt_pts_lst = re.findall('<span.*?>(.*?)</span>',pts_qt_pts_str)
pts_qt_pts_lst = [int(x) if x else None for x in pts_qt_pts_lst]

In [26]:
hometeam_pts_qtr = pts_qt_pts_lst[0::2]
awayteam_pts_qtr = pts_qt_pts_lst[1::2]

print(hometeam_name,hometeam_score,hometeam_pts_qtr)
print(awayteam_name,awayteam_score,awayteam_pts_qtr)

广东 107 [23, 27, 32, 25, None, None, None]
辽宁 98 [22, 25, 21, 30, None, None, None]


###  主队统计数据

### Todo:
* 从game_schedule表内加上队伍名称，比赛ID
* save as json?? df_schedule_full[:3].to_json(force_ascii=False,orient='index')

In [27]:
def get_team_details(page_content_str,re_pattern):
    team_details = re.findall(re_pattern,page_content_str)[0]
    return team_details

def get_table_coach(team_details):
    coach = re.findall('主教练：(.*?)领队',team_details)[0].replace('\xa0','')
    return coach

def get_table_lingdui(team_details):
    lingdui = re.findall('领队：(.*?)<',team_details)[0].replace('\xa0','')
    return lingdui

def get_table_headers(team_details):
    team_table_str = re.findall('<table>(.*?)</table>',team_details)[0]
    team_table_header_str = re.findall('<thead>(.*?)</thead>',team_table_str)[0]
    team_table_header_lst = re.findall('<th>(.*?)</th>',team_table_header_str)
    return team_table_header_lst

def clean_cells(cell_data):
    if '<a' in cell_data:
        cell_data = re.findall('<a.*?>(.*?)</a>',cell_data)[0]
    if 'document.write' in cell_data:
        try:
            cell_data = re.findall('[(]"(\d*)"[)]',cell_data)[0]
        except IndexError:
            cell_data = re.findall('[(]"(\d*/\d)"[)]',cell_data)[0]
    cell_data = re.sub('\s','', cell_data)
    cell_data = re.sub('是','1', cell_data)
    cell_data = re.sub('否','0', cell_data)
    return cell_data

def get_table_cells(team_details,headers_count):
    team_table_str = re.findall('<table>(.*?)</table>',team_details)[0]
    team_table_cells_str = re.findall('<tbody style="">(.*?)</tbody>',team_table_str)[0]
    team_table_cells_lst = re.findall('<td>(.*?)</td>',team_table_cells_str)
    team_table_cells_lst = list(map(clean_cells,team_table_cells_lst))
    team_table_cells_lst = np.reshape(team_table_cells_lst,(-1,headers_count)).tolist()
    return team_table_cells_lst

In [28]:
def get_hometeam_stats(page_content_str,re_pattern=
                       '<div class="part part01 blk">(.*?)<div class="part part02 blk">'):
    hometeam_details = get_team_details(page_content_str,re_pattern)

    hometeam_coach = get_table_coach(hometeam_details)
    hometeam_lingdui = get_table_lingdui(hometeam_details)
    
    hometeam_table_header_lst = get_table_headers(hometeam_details)
    hometeam_table_cells_lst = get_table_cells(hometeam_details,len(hometeam_table_header_lst))

    df_hometeam_details = pd.DataFrame(data = hometeam_table_cells_lst,columns=hometeam_table_header_lst)
    return df_hometeam_details

In [29]:
df_hometeam_details = get_hometeam_stats(page_content_str)
df_hometeam_details

,球员,号码,出场时间,首发,2分中-投,3分中-投,罚球中-投,进攻篮板,防守篮板,助攻,犯规,抢断,失误,盖帽,扣篮,被侵,快攻,得分
0,布鲁克斯,6,36,1,8-10(80%),2-10(20%),5-5(100%),0,3,9,3,2,1,0,1,5,0/0,27
1,任骏飞,20,30,1,5-8(63%),3-6(50%),0-0(0%),1,1,1,3,3,0,0,1,1,0/0,19
2,易建联,9,35,1,7-8(88%),0-0(0%),2-4(50%),1,11,2,2,1,1,2,5,3,0/0,16
3,赵睿,10,31,1,5-9(56%),0-3(0%),0-1(0%),2,5,4,3,2,0,0,0,4,0/0,10
4,胡明轩,3,13,1,0-1(0%),1-3(33%),0-0(0%),0,1,0,1,0,1,0,0,0,0/0,3
5,威姆斯,13,31,0,5-9(56%),1-5(20%),1-2(50%),0,7,3,2,1,3,0,0,3,0/0,14
6,杜润旺,18,14,0,2-2(100%),0-2(0%),2-2(100%),2,0,0,4,0,4,0,0,1,0/0,6
7,曾繁日,22,11,0,2-3(67%),0-0(0%),2-4(50%),1,0,0,3,0,0,1,1,3,0/0,6
8,徐杰,2,20,0,0-1(0%),1-3(33%),2-2(100%),0,0,1,2,2,0,0,0,3,0/0,5
9,王薪凯,5,9,0,0-0(0%),0-1(0%),1-2(50%),0,1,1,1,0,0,0,0,0,0/0,1


###  客队统计数据

In [30]:
def get_awayteam_stats(page_content_str,re_pattern=
                       '<div class="part part02 blk">(.*?)<div class="part part03 blk">'):
    awayteam_details = get_team_details(page_content_str,re_pattern)

    awayteam_coach = get_table_coach(awayteam_details)
    awayteam_lingdui = get_table_lingdui(awayteam_details)
    
    awayteam_table_header_lst = get_table_headers(awayteam_details)
    awayteam_table_cells_lst = get_table_cells(awayteam_details,len(awayteam_table_header_lst))

    df_awayteam_details = pd.DataFrame(data = awayteam_table_cells_lst,columns=awayteam_table_header_lst)
    return df_awayteam_details



In [31]:
df_awayteam_details = get_awayteam_stats(page_content_str)
df_awayteam_details

,球员,号码,出场时间,首发,2分中-投,3分中-投,罚球中-投,进攻篮板,防守篮板,助攻,犯规,抢断,失误,盖帽,扣篮,被侵,快攻,得分
0,巴斯,30,30,1,5-15(33%),0-0(0%),10-11(91%),3,9,1,2,1,0,2,2,10,0/0,20
1,郭艾伦,13,30,1,4-8(50%),0-2(0%),4-4(100%),0,4,5,4,0,6,0,0,2,0/0,12
2,韩德君,55,24,1,4-6(67%),0-0(0%),2-3(67%),3,6,0,4,0,2,1,0,2,0/0,10
3,刘志轩,11,16,1,1-1(100%),1-1(100%),0-0(0%),1,1,0,3,0,1,0,0,1,0/0,5
4,赵继伟,3,33,1,1-2(50%),0-3(0%),2-2(100%),0,3,1,2,2,0,0,0,2,0/0,4
5,史蒂芬森,1,41,0,10-17(59%),3-11(27%),5-7(71%),1,2,3,2,2,3,0,1,6,0/0,34
6,丛明晨,10,12,0,0-0(0%),2-2(100%),0-0(0%),1,2,0,1,0,0,0,0,0,0/0,6
7,李晓旭,22,28,0,3-8(38%),0-0(0%),0-0(0%),2,2,2,0,1,0,1,0,2,0/0,6
8,贺天举,6,12,0,0-0(0%),0-2(0%),1-2(50%),2,0,0,2,0,1,0,0,1,0/0,1
9,刘雁宇,17,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,0,0,0,0,0,0,0/0,0


In [32]:
pd.concat([df_hometeam_details,df_awayteam_details],ignore_index=True)

,球员,号码,出场时间,首发,2分中-投,3分中-投,罚球中-投,进攻篮板,防守篮板,助攻,犯规,抢断,失误,盖帽,扣篮,被侵,快攻,得分
0,布鲁克斯,6,36,1,8-10(80%),2-10(20%),5-5(100%),0,3,9,3,2,1,0,1,5,0/0,27
1,任骏飞,20,30,1,5-8(63%),3-6(50%),0-0(0%),1,1,1,3,3,0,0,1,1,0/0,19
2,易建联,9,35,1,7-8(88%),0-0(0%),2-4(50%),1,11,2,2,1,1,2,5,3,0/0,16
3,赵睿,10,31,1,5-9(56%),0-3(0%),0-1(0%),2,5,4,3,2,0,0,0,4,0/0,10
4,胡明轩,3,13,1,0-1(0%),1-3(33%),0-0(0%),0,1,0,1,0,1,0,0,0,0/0,3
5,威姆斯,13,31,0,5-9(56%),1-5(20%),1-2(50%),0,7,3,2,1,3,0,0,3,0/0,14
6,杜润旺,18,14,0,2-2(100%),0-2(0%),2-2(100%),2,0,0,4,0,4,0,0,1,0/0,6
7,曾繁日,22,11,0,2-3(67%),0-0(0%),2-4(50%),1,0,0,3,0,0,1,1,3,0/0,6
8,徐杰,2,20,0,0-1(0%),1-3(33%),2-2(100%),0,0,1,2,2,0,0,0,3,0/0,5
9,王薪凯,5,9,0,0-0(0%),0-1(0%),1-2(50%),0,1,1,1,0,0,0,0,0,0/0,1


###  各项最高 (Later)

In [33]:
highest_details = re.findall('<div class="part part01 blk">(.*?)<div class="part part02 blk">'
                            ,page_content_str)[0]



# Loop and Scrape

In [34]:
df_schedule_full[:3]

,轮次,日期,主队,比分,客队,战报,统计,组图,地点,电视,主队_link,比分_link,客队_link,战报_link,统计_link,组图_link,UID
0,1,2019-11-01 19:35,广东,107:98,辽宁,战报,统计,组图,东莞,,http://cba.sports.sina.com.cn/cba/team/show/1/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/9/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_79...,2019202000001
1,1,2019-11-02 19:35,江苏,96:97,浙江,战报,统计,组图,苏州,,http://cba.sports.sina.com.cn/cba/team/show/2/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/6/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000002
2,1,2019-11-02 19:35,四川,87:134,新疆,战报,统计,组图,成都,,http://cba.sports.sina.com.cn/cba/team/show/182/,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://cba.sports.sina.com.cn/cba/team/show/11/,http://sports.sina.com.cn/basketball/cba/2019-...,http://cba.sports.sina.com.cn/cba/schedule/sho...,http://slide.sports.sina.com.cn/cba/slide_2_78...,2019202000003


In [35]:
df_schedule_full.columns

Index(['轮次', '日期', '主队', '比分', '客队', '战报', '统计', '组图', '地点', '电视', '主队_link',
       '比分_link', '客队_link', '战报_link', '统计_link', '组图_link', 'UID'],
      dtype='object')

In [36]:
df_list = []
for index,row in df_schedule_full.iterrows():
    game_UID = row['UID']
    game_hteam_name = row['主队']
    game_ateam_name = row['客队']
    base_url = row['统计_link']
    page_content_str = get_page_content(base_url,encoding='GB2312')
    
    # Get home team stats
    df_home = get_hometeam_stats(page_content_str=page_content_str)
    df_home['Game_ID'] = game_UID
    df_home['球队'] = game_hteam_name
    df_home['对手'] = game_ateam_name
    df_list.append(df_home)
    
    # Get away team stats
    df_away = get_awayteam_stats(page_content_str=page_content_str)
    df_away['Game_ID'] = game_UID
    df_away['球队'] = game_ateam_name
    df_away['对手'] = game_hteam_name # use to get opponents stats
    df_list.append(df_away)
    print(row['UID'],' ',row['主队'],' ', row['客队'],' ',datetime.datetime.now())
    
    # pause for a few seconds to avoid getting banned
    time.sleep(np.random.rand()*10)

2019202000001   广东   辽宁   2020-01-07 16:24:25.421856
2019202000002   江苏   浙江   2020-01-07 16:24:30.402606
2019202000003   四川   新疆   2020-01-07 16:24:31.658143
2019202000004   山东   八一   2020-01-07 16:24:35.868352
2019202000005   北控   深圳   2020-01-07 16:24:42.314473
2019202000006   青岛   吉林   2020-01-07 16:24:51.740971
2019202000007   广厦   福建   2020-01-07 16:24:52.694661
2019202000008   上海   同曦   2020-01-07 16:24:56.044284
2019202000009   天津   北京   2020-01-07 16:25:04.700879
2019202000010   广州   山西   2020-01-07 16:25:05.830160
2019202000011   辽宁   广厦   2020-01-07 16:25:07.788928
2019202000012   北控   上海   2020-01-07 16:25:10.571090
2019202000013   同曦   江苏   2020-01-07 16:25:21.327156
2019202000014   山西   福建   2020-01-07 16:25:25.435575
2019202000015   青岛   八一   2020-01-07 16:25:35.877739
2019202000016   新疆   浙江   2020-01-07 16:25:38.718113
2019202000017   广东   四川   2020-01-07 16:25:48.414367
2019202000018   北京   山东   2020-01-07 16:25:49.277450
2019202000019   天津   吉林   2020-01-07 16:25:51.

In [37]:
games_stats = pd.concat(df_list,ignore_index=True)

games_stats

,球员,号码,出场时间,首发,2分中-投,3分中-投,罚球中-投,进攻篮板,防守篮板,助攻,...,抢断,失误,盖帽,扣篮,被侵,快攻,得分,Game_ID,球队,对手
0,布鲁克斯,6,36,1,8-10(80%),2-10(20%),5-5(100%),0,3,9,...,2,1,0,1,5,0/0,27,2019202000001,广东,辽宁
1,任骏飞,20,30,1,5-8(63%),3-6(50%),0-0(0%),1,1,1,...,3,0,0,1,1,0/0,19,2019202000001,广东,辽宁
2,易建联,9,35,1,7-8(88%),0-0(0%),2-4(50%),1,11,2,...,1,1,2,5,3,0/0,16,2019202000001,广东,辽宁
3,赵睿,10,31,1,5-9(56%),0-3(0%),0-1(0%),2,5,4,...,2,0,0,0,4,0/0,10,2019202000001,广东,辽宁
4,胡明轩,3,13,1,0-1(0%),1-3(33%),0-0(0%),0,1,0,...,0,1,0,0,0,0/0,3,2019202000001,广东,辽宁
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9565,刁成灏,24,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9566,孙椿棚,13,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9567,黄毅超,14,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9568,汤杰,7,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛


In [38]:
# remove team stats
if 2*len(games_stats['Game_ID'].unique())==len(games_stats.loc[games_stats['号码']=='--']):
    games_stats.drop(games_stats.loc[games_stats['号码']=='--'].index,inplace=True)

def split_made_attempt(df_orig):
    df = df_orig.copy()
    for col_name in list(filter(lambda x: '-' in x, df.columns.tolist())):
        orig_col = col_name
        col_made = re.findall('(.*)中-投',col_name)[0]+'中'
        col_attempt = re.findall('(.*)中-投',col_name)[0]+'投'
        df[[col_made,col_attempt]] = df[orig_col].str.split('-',expand=True)
        df[col_attempt] = df[col_attempt].apply(lambda x: re.sub('[(].*[)]','',x))
        df.drop(columns=orig_col,inplace=True)
    return df

games_stats = split_made_attempt(games_stats)
games_stats

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,得分,Game_ID,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投
0,布鲁克斯,6,36,1,0,3,9,3,2,1,...,27,2019202000001,广东,辽宁,8,10,2,10,5,5
1,任骏飞,20,30,1,1,1,1,3,3,0,...,19,2019202000001,广东,辽宁,5,8,3,6,0,0
2,易建联,9,35,1,1,11,2,2,1,1,...,16,2019202000001,广东,辽宁,7,8,0,0,2,4
3,赵睿,10,31,1,2,5,4,3,2,0,...,10,2019202000001,广东,辽宁,5,9,0,3,0,1
4,胡明轩,3,13,1,0,1,0,1,0,1,...,3,2019202000001,广东,辽宁,0,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,胡珑贸,10,3,0,0,0,0,0,0,1,...,0,2019202000260,福建,青岛,0,0,0,1,0,0
9565,刁成灏,24,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0
9566,孙椿棚,13,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0
9567,黄毅超,14,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0


In [39]:
games_stats.to_csv(f'StatsData/All_Games_Stats_{datetime.date.today()}.csv',encoding = 'UTF-8',
                   index=False)